In [7]:
import os
import pandas as pd
from pyDataverse.api import NativeApi, DataAccessApi

# ==========================================
# CONFIGURACIÓN
# ==========================================

API_KEY = "7386f7e6-18cc-49ee-a8eb-d08ac888783d"
BASE_URL = "https://datospararesiliencia.cl"

DATA_DIR = "datos_eventos"
os.makedirs(DATA_DIR, exist_ok=True)

DOIS = [
    "doi:10.71578/FMGXND",
    "doi:10.71578/UXAUN5",
    "doi:10.71578/QRDY49",
    "doi:10.71578/HKLGZ8",
    "doi:10.71578/TLSNSV",
]

VALID_EXT = {".csv", ".xlsx", ".xls"}


api = NativeApi(BASE_URL, API_KEY)
data_api = DataAccessApi(BASE_URL, API_KEY)


# ==========================================
# UTILIDADES
# ==========================================

# --- 1: corregir caracteres maldecodificados ---
def fix_encoding(col):
    try:
        return col.encode("latin1").decode("utf8")
    except:
        return col


# Detectar columna Región
def detectar_columna_region(df):
    for col in df.columns:
        c = col.lower().strip()
        if "regi" in c:
            return col
    return None


# Detectar columna Fecha
def detectar_columna_fecha(df):
    posibles = ["fecha", "fecha_update"]
    for col in df.columns:
        if col.lower().strip() in posibles:
            return col
    return None


# Normalizar nombre de región final
def normalizar_region(valor):
    v = str(valor).lower()
    if "bio" in v:
        return "BioBio"
    if "nuble" in v:
        return "Ñuble"
    return None


# ==========================================
# API: descargar archivos
# ==========================================
def download_dataset_files(doi):
    print("\n===================================")
    print("Procesando dataset:", doi)
    print("===================================\n")

    ds = api.get_dataset(doi)

    if ds.status_code != 200:
        print("Error obteniendo dataset:", doi)
        return []

    files = ds.json()["data"]["latestVersion"]["files"]
    downloaded_paths = []

    for f in files:
        fname = f["dataFile"]["filename"]
        fid = f["dataFile"]["id"]
        ext = os.path.splitext(fname)[1].lower()

        print(f" Archivo: {fname}, id={fid}")

        if ext not in VALID_EXT:
            print("   → No tabular, omitido.")
            continue

        resp = data_api.get_datafile(fid, is_pid=False)
        if resp.status_code != 200:
            print("Error descargando", fname)
            continue

        path = os.path.join(DATA_DIR, fname)
        with open(path, "wb") as f:
            f.write(resp.content)

        print("Guardado en", path)
        downloaded_paths.append(path)

    return downloaded_paths


# ==========================================
# PROCESAR Y LIMPIAR TABLAS
# ==========================================
def load_and_clean_file(path):
    try:
        # -------------------------
        # 1. Cargar archivo
        # -------------------------
        if path.endswith(".csv"):
            df = pd.read_csv(path, encoding="latin-1")
        elif path.endswith(".xlsx") or path.endswith(".xls"):
            df = pd.read_excel(path, engine="openpyxl")
        else:
            return None

        # -------------------------
        # 2. Corregir encoding de columnas
        # -------------------------
        df.columns = [fix_encoding(c) for c in df.columns]

        # -------------------------
        # 3. Separar columnas fusionadas con "|"
        # -------------------------
        cols_to_split = [c for c in df.columns if "|" in c and df[c].notna().sum() > 0]
        split_frames = []

        for c in cols_to_split:
            print(f"→ Separando columna fusionada: {c}")
            new_cols = c.split("|")
            expanded = df[c].astype(str).str.split("|", expand=True)

            if expanded.shape[1] == len(new_cols):
                expanded.columns = new_cols
                split_frames.append(expanded)
                df = df.drop(columns=[c])

        for f in split_frames:
            df = pd.concat([df, f], axis=1)

        # -------------------------
        # 4. Detectar columnas útiles
        # -------------------------
        col_region = detectar_columna_region(df)
        if col_region is None:
            print(f"Omitido (sin columna región): {path}")
            return None

        col_fecha = detectar_columna_fecha(df)
        if col_fecha is None:
            print(f"Omitido (sin columna fecha): {path}")
            return None

        # -------------------------
        # 5. Normalizar región
        # -------------------------
        df[col_region] = (
            df[col_region]
            .astype(str)
            .apply(fix_encoding)      # corregir caracteres maldecodificados
            .str.normalize("NFKD")    # normalizar tildes
            .str.encode("ascii", "ignore")  # convertir Ñ -> n si fuera necesario
            .str.decode("utf8")
            .str.lower()
            .str.strip()
        )

        # -------------------------
        # 6. Filtrar solo regiones relevantes
        # -------------------------
        mask = df[col_region].str.contains("bio") | df[col_region].str.contains("nuble")
        df = df[mask]

        if df.empty:
            print(f"   ⚠ Archivo sin filas de Biobío/Ñuble: {path}")
            return None

        # -------------------------
        # 7. Convertir la fecha a formato YYYY-MM-DD
        # -------------------------
        df[col_fecha] = pd.to_datetime(df[col_fecha], errors="coerce")

        # eliminar filas sin fecha válida
        df = df.dropna(subset=[col_fecha])

        if df.empty:
            print(f"   ⚠ Archivo sin fechas válidas después de normalizar: {path}")
            return None

        df[col_fecha] = df[col_fecha].dt.strftime("%Y-%m-%d")

        # -------------------------
        # 8. Normalizar nombres finales de región
        # -------------------------
        df["Region"] = df[col_region].apply(normalizar_region)
        df["Fecha"] = df[col_fecha]

        df = df[["Region", "Fecha"]]

        return df

    except Exception as e:
        print("Error leyendo/limpiando", path, e)
        return None


# ==========================================
# PROCESAR TODOS LOS DATASETS
# ==========================================

all_clean_dfs = []

for doi in DOIS:
    paths = download_dataset_files(doi)

    for path in paths:
        df_clean = load_and_clean_file(path)
        if df_clean is not None:
            all_clean_dfs.append(df_clean)

if not all_clean_dfs:
    raise RuntimeError("No se pudo cargar ningún archivo válido con región y fecha.")

# ==========================================
# UNIFICAR
# ==========================================

df_final = pd.concat(all_clean_dfs, ignore_index=True)

output_path = os.path.join(DATA_DIR, "eventos_unificados.csv")
df_final.to_csv(output_path, index=False)

print("\n===================================")
print("ARCHIVO FINAL GENERADO")
print("===================================")
print("Dimensiones finales:", df_final.shape)
print(df_final.head())
print(f"\nGuardado en: {output_path}")



Procesando dataset: doi:10.71578/FMGXND

 Archivo: 2015.csv, id=302
Guardado en datos_eventos\2015.csv
 Archivo: 2016.csv, id=303
Guardado en datos_eventos\2016.csv
 Archivo: 2017.csv, id=304
Guardado en datos_eventos\2017.csv
 Archivo: 2018.csv, id=305
Guardado en datos_eventos\2018.csv
 Archivo: - Archivo Indice.csv, id=306
Guardado en datos_eventos\- Archivo Indice.csv
Omitido (sin columna región): datos_eventos\- Archivo Indice.csv

Procesando dataset: doi:10.71578/UXAUN5



C:\Users\mate\AppData\Local\Temp\ipykernel_21852\1018215776.py:192: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col_fecha] = pd.to_datetime(df[col_fecha], errors="coerce")
C:\Users\mate\AppData\Local\Temp\ipykernel_21852\1018215776.py:192: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col_fecha] = pd.to_datetime(df[col_fecha], errors="coerce")


 Archivo: 20022003.csv, id=5234
Guardado en datos_eventos\20022003.csv
 Archivo: 20022003.geojson, id=5238
   → No tabular, omitido.
 Archivo: 20022003.zip, id=5231
   → No tabular, omitido.
 Archivo: 20032004.csv, id=5434
Guardado en datos_eventos\20032004.csv
 Archivo: 20032004.geojson, id=5436
   → No tabular, omitido.
 Archivo: 20032004.zip, id=5430
   → No tabular, omitido.
 Archivo: 20042005.csv, id=5225
Guardado en datos_eventos\20042005.csv
 Archivo: 20042005.geojson, id=5228
   → No tabular, omitido.
 Archivo: 20042005.zip, id=5223
   → No tabular, omitido.
 Archivo: 20052006.csv, id=5375
Guardado en datos_eventos\20052006.csv
 Archivo: 20052006.geojson, id=5378
   → No tabular, omitido.
 Archivo: 20052006.zip, id=5363
   → No tabular, omitido.
 Archivo: 20062007.csv, id=5300
Guardado en datos_eventos\20062007.csv
 Archivo: 20062007.geojson, id=5304
   → No tabular, omitido.
 Archivo: 20062007.zip, id=5298
   → No tabular, omitido.
 Archivo: 20072008.csv, id=5416
Guardado en d

In [18]:
import pandas as pd
import unicodedata

def fix_encoding(text):
    if not isinstance(text, str):
        return text
    try:
        return text.encode("latin1").decode("utf8")
    except:
        return text

# -----------------------------------------
# 1. Cargar archivo
# -----------------------------------------
input_file = "inversion_semestres.xlsx"
df = pd.read_excel(input_file)

# -----------------------------------------
# 2. Intentar detectar columna Región
# -----------------------------------------
def normalize_for_match(s):
    """Normaliza texto eliminando tildes y bajando a minúsculas."""
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

normalized_cols = {normalize_for_match(c): c for c in df.columns}

possible_region_keys = [
    "region",
    "región",
    "region administrativa",
    "nombre region",
    "nombre región",
]

col_region = None
for key in possible_region_keys:
    key_norm = normalize_for_match(key)
    if key_norm in normalized_cols:
        col_region = normalized_cols[key_norm]
        break

if col_region is None:
    print("\nNo se encontró la columna Región. Columnas disponibles:")
    for c in df.columns:
        print(" -", c)
    raise KeyError("No existe columna equivalente a 'Región' en el archivo.")

print("Columna Región detectada como:", col_region)

# -----------------------------------------
# 3. Detectar columna Fecha
# -----------------------------------------
possible_fecha_keys = ["fecha", "date", "fechas"]

col_fecha = None
for key in possible_fecha_keys:
    key_norm = normalize_for_match(key)
    if key_norm in normalized_cols:
        col_fecha = normalized_cols[key_norm]
        break

if col_fecha is None:
    raise KeyError("No se encontró una columna de fecha.")

print("Columna Fecha detectada como:", col_fecha)

# -----------------------------------------
# 4. Detectar columna Inversión
# -----------------------------------------
possible_inv_keys = [
    "inversion",
    "inversión",
    "inversion (miles de $ de cada año)",
    "monto inversion",
    "monto",
]

col_inv = None
for key in possible_inv_keys:
    key_norm = normalize_for_match(key)
    if key_norm in normalized_cols:
        col_inv = normalized_cols[key_norm]
        break

if col_inv is None:
    print("\nNo se encontró la columna de inversión. Columnas disponibles:")
    for c in df.columns:
        print(" -", c)
    raise KeyError("No existe columna de inversión en el archivo.")

print("Columna Inversión detectada como:", col_inv)

# -----------------------------------------
# 5. Normalizar Región
# -----------------------------------------
df[col_region] = (
    df[col_region]
    .astype(str)
    .apply(fix_encoding)
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf8")
    .str.lower()
    .str.strip()
)

# Variantes aceptadas
bio_vars = ["bio bio", "biobio", "bio-bio"]
nuble_vars = ["nuble", "ñuble"]

df_filtered = df[df[col_region].isin(bio_vars + nuble_vars)].copy()

# Mapear al nombre final
def map_region(r):
    if r in bio_vars:
        return "BioBio"
    if r in nuble_vars:
        return "Ñuble"
    return None

df_filtered["Region"] = df_filtered[col_region].apply(map_region)

# -----------------------------------------
# Normalizar fecha
# -----------------------------------------
df_filtered["Fecha"] = pd.to_datetime(df_filtered[col_fecha], dayfirst=True, errors="coerce").dt.date

# -----------------------------------------
# Normalizar inversión
# -----------------------------------------
df_filtered["Inversion"] = (
    df_filtered[col_inv]
    .astype(str)
    .str.replace(r"[^0-9,.\-]", "", regex=True)
    .str.replace(",", ".", regex=False)
    .astype(float)
    .mul(1000)
    .round(0)        # redondea al entero más cercano
    .astype(int)     # convierte a entero
)

# -----------------------------------------
# Exportar CSV
# -----------------------------------------
output_file = "inversion_filtrada.csv"
df_filtered[["Region", "Fecha", "Inversion"]].to_csv(output_file, index=False)

print("\nArchivo generado correctamente:", output_file)


Columna Región detectada como: REGIÓN
Columna Fecha detectada como: Fecha
Columna Inversión detectada como: INVERSIÓN (MILES DE $ DE CADA AÑO)

Archivo generado correctamente: inversion_filtrada.csv
